In [ ]:
!pip install lightning

In [3]:
import lightning

In [5]:
import torch
import torchvision.models.detection.faster_rcnn
import torchvision.ops
import torch.optim

In [47]:
def _evaluate_iou(target, pred):
    if pred["boxes"].shape[0] == 0:
        return torch.tensor(0.0, device=pred["boxes"].device)
    return torchvision.ops.box_iou(target["boxes"], pred["boxes"]).diag().mean()


class Frcnn(lightning.LightningModule):

    def __init__(self, learning_rate: float):
        super().__init__()
        self.model = torchvision.models.detection.faster_rcnn.fasterrcnn_resnet50_fpn(pretrained=True)
        self.learning_rate = learning_rate

    def forward(self, x):
        self.model.eval()
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        targets = [{k: v for k, v in t.items()} for t in targets]
        loss_dict = self.model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        return {"loss": loss, "log": loss_dict}
        
    def validation_step(self, batch, batch_idx):
        images, targets = batch
        self.model.eval()
        outs = self.model(images)
        iou = torch.stack([_evaluate_iou(t, o) for t, o in zip(targets, outs)]).mean()
        return {"val_iou": iou}

    #def validation_epoch_end(self, outs):
    #    avg_iou = torch.stack([o["val_iou"] for o in outs]).mean()
    #    logs = {"val_iou": avg_iou}
    #    return {"avg_val_iou": avg_iou, "log": logs}

    def configure_optimizers(self):
        return torch.optim.SGD(
            self.model.parameters(),
            lr=self.learning_rate,
            momentum=0.9,
            weight_decay=0.005,
        )


In [43]:
import os
import typing
import torch
import torchvision
import numpy
import pandas
import collections


LabeledImageSample = collections.namedtuple('LabeledImageSample', ['image', 'labels'])
xf = ['cx', 'w']
yf = ['cy', 'h']


class Dataset(torch.utils.data.Dataset):

    def __init__(self, folder: str, num=None) -> None:
        self.__folder = folder
        self.__image = {}
        self.__bbox = {}
        path = os.path.join(folder, 'labels.txt')
        self.meta = pandas.read_csv(path, sep=',')
        self.image_files = self.meta['image'].unique()
        self.__num = num

    def __len__(self) -> int:
        if self.__num is not None:
            return self.__num

        return len(self.image_files)

    def __getitem__(self, item: int) -> pandas.DataFrame:
        file = self.image_files[item]
        index = self.meta['image'] == file
        if file not in self.__image:
            self.__image[file] = torchvision.io.read_image(os.path.join(
                self.__folder, file))
            self.meta.loc[index, xf] = self.meta.loc[
                index, xf]*self.__image[file].shape[2]
            self.meta.loc[index, yf] = self.meta.loc[
                index, yf]*self.__image[file].shape[1]
            self.meta['p1x'] = self.meta['cx'] - self.meta['w']/2
            self.meta['p2x'] = self.meta['cx'] + self.meta['w']/2
            self.meta['p1y'] = self.meta['cy'] - self.meta['h']/2
            self.meta['p2y'] = self.meta['cy'] + self.meta['h']/2

        item = LabeledImageSample(image=self.__image[file],
                                  labels=self.meta.loc[index])

        inputs = item.image.float()/255
        labels = {
            'boxes': torch.from_numpy(
                item.labels[['p1x', 'p1y', 'p2x', 'p2y']].to_numpy()
                ),
            'labels': torch.from_numpy(
                item.labels['label'].to_numpy())
        }
        if len(inputs.shape) < 4:
            inputs = inputs.reshape(-1, 
                                    inputs.shape[0],
                                    inputs.shape[1],
                                    inputs.shape[2])
            labels = [labels]

        return inputs, labels

In [25]:
!ls drive/MyDrive/neuro_pract2/

train.zip  validation.zip


In [ ]:
!unzip -d train drive/MyDrive/neuro_pract2/train.zip
!unzip -d validate drive/MyDrive/neuro_pract2/validation.zip

In [48]:
model = Frcnn(learning_rate=0.01)
trainer = lightning.Trainer()
trainer.fit(model, Dataset(folder='train'), Dataset(folder='validate'))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
